In [115]:
import os
import pandas as pd
sentiment_path = os.path.join(os.path.pardir,'data','raw','algorithms','naive-bayes')
data_path = os.path.join(sentiment_path,'fromUnivCalifornia')
amazon_txt_path = os.path.join(data_path,'amazon_cells_labelled.txt')
imdb_txt_path = os.path.join(data_path,'imdb_labelled.txt')
yelp_txt_path = os.path.join(data_path,'yelp_labelled.txt')
amazon_data = pd.read_table(amazon_txt_path,header=None,names=['sentence','posneg']) 
imdb_data = pd.read_table(imdb_txt_path,header=None,names=['sentence','posneg']) 
yelp_data = pd.read_table(yelp_txt_path,header=None,names=['sentence','posneg']) 
# yelp_data[yelp_data.isnull().any(axis=1)]
# yelp_data.isnull().any()
imdb_data.iloc[19]

sentence     The structure of this film is easily the most...
posneg                                                      0
Name: 19, dtype: object

In [116]:
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
sentence    748 non-null object
posneg      748 non-null int64
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [117]:
total_data = amazon_data.append(imdb_data,ignore_index=True).append(yelp_data,ignore_index=True)
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
sentence    2748 non-null object
posneg      2748 non-null int64
dtypes: int64(1), object(1)
memory usage: 43.0+ KB


In [118]:
total_data.tail()

,sentence,posneg
2743,I think food should have flavor and texture an...,0
2744,Appetite instantly gone.,0
2745,Overall I was not impressed and would not go b...,0
2746,"The whole experience was underwhelming, and I ...",0
2747,"Then, as if I hadn't wasted enough of my life ...",0


In [122]:
train_doc = total_data.drop('posneg', axis=1)
train_label = total_data.drop('sentence', axis=1)

In [127]:
train_doc.tail()
train_label.tail()

,posneg
2743,0
2744,0
2745,0
2746,0
2747,0


In [135]:
# Data processing phase
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(binary='true')
train_doc_temp = count_vectorizer.fit_transform(train_doc['sentence'])
train_doc_temp

<2748x5155 sparse matrix of type '<class 'numpy.int64'>'
	with 30275 stored elements in Compressed Sparse Row format>

In [137]:
count_vectorizer.vocabulary_.get('great') # to get the id of a certain word

2023

In [138]:
pd.DataFrame(train_doc_temp.toarray(),columns=count_vectorizer.get_feature_names()).head()

,00,10,100,11,12,13,15,15g,15pm,17,...,yucky,yukon,yum,yummy,yun,z500a,zero,zillion,zombie,zombiez
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
# Training Phase
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB().fit(train_doc_temp, train_label['posneg'])

In [129]:
# Test Phase
classifier.predict(count_vectorizer.transform(["this is the best movie I've ever seen!"]))

array([1])

In [131]:
classifier.predict(count_vectorizer.transform(["totally uninterested"]))

array([0])